<a href="https://colab.research.google.com/github/nhwhite212/DealingwithDataSpring2021/blob/master/6-Pandas/E-Citibike_Mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Plotting Real-time Data from Citibike

We will now use the Citibike API in combination with Pandas, in order to create a real-time view of the Citibike stations in New York.

The API call at http://www.citibikenyc.com/stations/json gives us a real-time view of the Citibike stations in New York.

In [ ]:
# Plotting setup

%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (20, 20)
matplotlib.style.use(['seaborn-talk', 'seaborn-ticks', 'seaborn-whitegrid'])

In [ ]:
import requests
url = 'http://www.citibikenyc.com/stations/json'
data = requests.get(url).json()
data

The JSON results contain two keys: The `executionTime` and `stationBeanList`. The `stationBeanList` is a list of dictionaries, with each dictionary corresponding to a Citibike station.

In [ ]:
data.keys()

Pandas can directly convert a list of dictionaries into a dataframe.

In [ ]:
import pandas
df = pandas.DataFrame(data["stationBeanList"])
df.head(5)

### Plotting

We can now create a few plots that will give us a feeling of the status of Citibike stations.

First a histogram of the sizes of citibike stations:

In [ ]:
df["totalDocks"].hist(alpha=0.75)

And a histogram of the available bikes across bike stations (real-time):

In [ ]:
df["availableBikes"].hist(alpha=0.75)

Let's create a variable that shows the percent of capacity at each station.

In [ ]:
df["perc_full"] = df['availableBikes']/df['totalDocks']

In [ ]:
df["perc_full"].hist(alpha=0.5, bins=20)

#### Poor man's mapping

We will now do a small attempt at using a scatterplot, together with the longitude and latitude coordinates to plot things in a map.

In [ ]:
df.plot(
    kind='scatter', 
    x='longitude', 
    y='latitude',
    figsize = (10,10)
)

### Using multiple visual channels for plotting

We used the x and y axes in the plot to show the locations of the stations. 

When we visualize data, we can use multiple visual channels to communicate information. For example, we can use the size of the dot to communicate how full each station is.

In [ ]:
# The s parameter controls the size of the market. We make the size proportional to the perc_full column
df.plot(
    kind='scatter', 
    x='longitude', 
    y='latitude', 
    s = 100*df['perc_full'], alpha=0.75,
    figsize = (10,10)
)

### Using GeoPandas to plot map outlines

We can use GeoPandas to also plot the map underneath the "map" that we created above. For that, we will need to install the geopandas package:

In [ ]:
! pip3 install -U geopandas fiona shapely pyproj geopy pysal descartes

In [ ]:
!sudo apt-get install -y libgeos-dev python3-rtree

In [ ]:
import geopandas as gpd

To draw the maps, we need the corresponding **shapefile** that describes the shapes of the areas that we want to plot. For our example, we will use the shapefile of the NYC neighborhoods that is available from the NYC Open Data repository.

In [ ]:
# Dataset from NYC Open Data: https://data.cityofnewyork.us/City-Government/Neighborhood-Tabulation-Areas/cpf4-rkhq
!mkdir data
!rm data/nyc-neighborhoods.geojson
#!curl 'https://data.cityofnewyork.us/api/geospatial/cpf4-rkhq?method=export&format=GeoJSON' -o data/nyc-neighborhoods.geojson
!curl http://people.stern.nyu.edu/nwhite/DealingwithDataSpring2021/data/nyc-neighborhoods.geojson -o data/nyc-neighborhoods.geojson

We use the shapefile that we downloaded (which is in _GeoJSON_ format) and we create a "geo-dataframe". Lets take a peek at the contents.

In [ ]:
#!ls data/*#
#!head data/nyc-neighborhoods.geojson
#
df_nyc = gpd.GeoDataFrame.from_file('data/nyc-neighborhoods.geojson')

We can plot the geo information, using our usual syntax.

In [ ]:
df_nyc.plot(figsize=(10,10))

### Combining plots

We can combine plots by using the `ax` parameter, and pass one plot as parameter to the other. Below, we create the `base` plot, with the neighborhood outlines, and then we pass `base` as the `ax` parameter to the plot that draws the Citibike stations.

In [ ]:
base = df_nyc.plot(linewidth=0.3, color='grey', figsize = (20,20))

df.plot(
    kind='scatter', 
    x='longitude', 
    y='latitude', 
    s = 100*df['perc_full'],
    ax = base
)

### Exercise

* Use the NYC Vehicle Collisions dataset, and plot the accidents with at least one death in the NYC map.

In [ ]:
# your code here

### Beautifying the Plot

* Lets reduce the x- and y- range to focus on the areas for which we have data. We use the `set_xlim` and `set_ylim` for that purpose.
* We will use the `zorder` parameter to define which plot goes to the top, which to the bottom, etc. The element with the lowest zorder goes to the bottom, and the one with the highest zorder on top.

In [ ]:
base = df_nyc.plot(linewidth=0.3, color='grey', zorder=1, figsize = (20,20))
base.set_xlim( (-74.025,-73.92)  )
base.set_ylim( (40.65  , 40.81) )
df.plot(
    kind='scatter', 
    x='longitude', 
    y='latitude', 
    s = 100*df['perc_full'],
    ax = base, zorder=2
)

#### Adding more communication channels

Earlier, we used the size of the marker to communicate how full each station is. Now, we will use a different type of a marker, and a different color, to show stations that are out of service.

In [ ]:
inservice = df[df["statusValue"] == 'In Service']
outofservice = df[df["statusValue"] == 'Not In Service']
inservice.describe()
inservice.plot(kind='scatter',x='longitude',y='latitude',s=100*inservice['perc_full'])

In [ ]:
# Plot the map
base = df_nyc.plot(linewidth=0.3, color='lightgrey', edgecolor='black',zorder=1, figsize = (20,20))
base.set_xlim( (-74.025,-73.92)  )
base.set_ylim( (40.65  , 40.81) )
# Plot the in-service stations, and make the size of each station proportional to its fullness
inservice.plot(
    kind='scatter', 
    x='longitude', 
    y='latitude', 
    s = 100*inservice['perc_full'],
    ax = base, zorder=2, 
    label = 'In Service'
)
# Plots the out-of-service stations with a red x mark
outofservice.plot(
    kind='scatter',
    x='longitude',
    y='latitude',
    edgecolor = 'Black', linewidth=0.5,
    color='Red', marker = 'X',
    s = 100, alpha = 0.75, 
    ax = base, 
    label = 'Out of Service', zorder=3)

#### More advanced
#### Show more dimensions by altering circle sizes and transparency
The part below tries to add even more communication channels. We change the size of the circle to correspond to the size of the station, and then we use the color transparency to show how full (or empty) the station is. Unfortunately, due to limitations of Pandas, we cannot set directly the transparency of each data point, so we need to go and do the plotting directly using matplotlib functions. In that case, we set the color and transparency of each bubble directly.

In [ ]:
import numpy as np
fig, ax = plt.subplots()
ax.set_aspect('equal')

ax.set_xlim( (-74.025,-73.92)  )
ax.set_ylim( (40.66  , 40.81) )

# And now make the size of each point proportional to the size of available bikes
# using the s option in the plot
inservice = df[df["statusValue"] == 'In Service']
outofservice = df[df["statusValue"] == 'Not In Service']

# alpha is the transparency

#DRAW the background 

df_nyc.plot(linewidth=0.3, color='lightgrey',ax=ax, edgecolor='black',alpha = 0.5, zorder=1)

# Following http://stackoverflow.com/questions/26738881/setting-scatter-points-transparency-from-data-array-in-matplotlib-plot
# to set alpha in varying degrees
plt.scatter( 
    x=inservice['longitude'],
    y=inservice['latitude'],
# This is using the RGB model (i.e. Red=0,Green=0, Blue=1, alpha is the perc_full value)
# It is generating a list of {R,G,B,alpha) tuples to match each data point

    color=np.asarray([(0, 0, 1, a) for a in inservice['perc_full']]), 
    edgecolors='Black', 
    label='In Service',
    s=3*inservice['totalDocks'], 
    zorder=2
)
outofservice.plot(figsize = (20,20),
    kind='scatter',
    ax = ax, 
    x='longitude',
    y='latitude',
    edgecolor = 'Black', color='Red', marker = 'X',
    s = 200, alpha = 0.75,
    label = 'Out of Service', 
    zorder=3
)

In [ ]:
# Play with the sizes of the circles and the transparency. How do you change them?

In [ ]:
#your code here
